# 4 · The Harmonic Spectrometer

**Observational record associated with the book**  
*Discovering Chaos in Prime Numbers — Computational Investigations through the Euler Mirror*  
© Alvaro Costa, 2025  

This notebook is part of a canonical sequence of computational records.  
It does not introduce new hypotheses, conjectures, or interpretative models.

Its sole purpose is to **record** the behaviour of arithmetic structures under an  
explicit, deterministic, and reproducible regime of observation.

The complete conceptual discussion is contained in the book.  
This notebook documents only the corresponding experiment.

**Licence:** Creative Commons BY–NC–ND 4.0  
Reading, execution, and citation are permitted.  
Modification, adapted redistribution, or independent commercial use are not permitted.


---

## 1. From Signal to Structure: The Need for a New Dimension

In the previous chapter, we uncovered the “pulse” of the primes: a one-dimensional signal, $\Delta_\pi(x)$, measuring the tension  
between stabilising and structuring forces. This signal, although rich, is like the seismograph of a distant earthquake — it tells  
us that something is happening, but it does not reveal the full structure of the event.

To understand the “music” contained in this signal — its harmonies, resonances, and deep correlations — we need a more powerful  
instrument. We must project this 1D signal into a higher dimension.

Our aim in this chapter is to construct a **harmonic spectrometer**: a matrix $M$ that translates the arithmetic signal  
$\Delta_\pi(x)$ into a geometric structure, whose spectral properties (its eigenvalues and eigenvectors) can finally be analysed.

---

## 2. The Anatomy of the Operator $M$: The Ingredients

At first glance, the formula for our spectrometer may appear exotic — but each element has been chosen for a precise reason  
that connects number theory to physics:

$
M_{i,j} = \cos(\Delta_\pi(x_i) \cdot \ln(x_j)) + \cos(\Delta_\pi(x_j) \cdot \ln(x_i))
$

---

### Why the Logarithm? · The Scale of Nature

Prime numbers are, by nature, **multiplicative** entities.  
Their density, as indicated by the Prime Number Theorem ($\pi(x) \approx x / \ln x$), decays logarithmically.  
From the perspective of primes, the “natural” way to observe the number line is **not linear, but logarithmic**.

The $\ln(x)$ term acts as a lens that adjusts our view to the correct scale.  
It “straightens” the number line, placing numbers into a perspective where their internal relations become clearer and more stable.

---

### Why the Cosine? · Vibration and Measurement

The cosine is the fundamental function of waves, vibrations, and harmony.  
It performs two essential tasks for our instrument:

1. **Transforms Tension into Vibration** — The product $\Delta_\pi(x) \cdot \ln(x)$ is converted into an oscillation bounded  
   between $-1$ and $1$. The arithmetic tension is translated into harmonic phase.

2. **Measures Intensity** — The cosine is an even function ($\cos(\theta) = \cos(-\theta)$), meaning that the spectrometer  
   **does not distinguish sign**. It measures the intensity of oscillation rather than its direction — peaks and troughs are  
   treated symmetrically.

---

## 3. Symmetry · The Key to Observability

The way the terms are combined is the final piece of the construction.

We define the matrix as  
$
M_{ij} = C_{ij} + C_{ji}, \quad \text{where} \quad C_{ij} = \cos(\Delta_\pi(x_i)\ln(x_j)),
$
ensuring, by construction, that it is **symmetric** ($M = M^T$).

Why is this crucial?

In quantum physics, operators representing observable quantities (such as the energy of a system) are **Hermitian**.  
For real matrices, this is equivalent to symmetry.  
And a symmetric matrix has a fundamental property: **all of its eigenvalues are real**.

By constructing $M$ in this way, we ensure that its “energy spectrum” (the eigenvalues) lies entirely on the real line,  
making it suitable for rigorous spectral analysis. We are, in effect, constructing the **Hamiltonian of an arithmetic atom**,  
whose internal structure is dictated by the primes.

---

## 4. Laboratory · From Disorder to Harmony

The code cell below implements our spectrometer $M$ and generates a heatmap of its structure for different initial scales $X_0$.  
It allows the central claim of this work to be examined directly.

**What to observe**:

- At **low $X_0$**, where the pulse $\Delta_\pi(x)$ is strong and irregular, the matrix appears **chaotic, noisy, and fragmented**.  
- At **high $X_0$**, where the pulse becomes damped and more subtle, **geometric patterns and visual harmony** emerge from the disorder.


In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# --- 1. OPTIMISED PRIME GENERATION (THE CORE ENGINE) ---
def generate_primes_upto(n: int) -> np.ndarray:
    """
    Generates an array containing all primes up to n
    using an optimised sieve for odd numbers.
    """
    if n < 2:
        return np.array([], dtype=np.int64)

    # Only odd numbers are represented
    size = (n - 1) // 2
    sieve = np.ones(size, dtype=bool)

    limit = int(np.sqrt(n)) // 2
    for i in range(limit):
        if sieve[i]:
            p = 2 * i + 3
            start = (p * p - 3) // 2
            sieve[start::p] = False

    # Estimate upper bound for number of primes (slight safety margin)
    primes = np.empty(int(n / np.log(n) * 1.15), dtype=np.int64)
    primes[0] = 2

    num_primes = 1 + np.sum(sieve)
    indices = np.where(sieve)[0]
    primes[1:num_primes] = 2 * indices + 3

    return primes[:num_primes]

# --- 2. VECTORISED COMPUTATION OF METRICS ---
def generate_pi_metrics(N_max, primes):
    """
    Computes all arithmetic metrics required for the harmonic operator,
    using fully vectorised NumPy operations.
    """
    print(f"Computing metrics up to N = {N_max:,}...")

    # Cumulative prime counting function pi(x)
    pi_cumulative = np.zeros(N_max + 1, dtype=np.int32)
    pi_cumulative[primes] = 1
    pi_cumulative = np.cumsum(pi_cumulative)

    # Compute all quantities at once
    x_range = np.arange(1, N_max + 1)
    pi_x = pi_cumulative[x_range]
    pi_S_x = pi_cumulative[x_range // 2]   # π(floor(x/2))
    pi_N_x = pi_x - pi_S_x
    delta_pi = pi_N_x - pi_S_x

    # The spectrometer uses the absolute arithmetic tension
    F_x = np.abs(delta_pi)

    # Package results for downstream use
    metrics = {
        'x': x_range,
        'delta_pi': delta_pi,
        'F_x': F_x
    }

    return metrics


In [2]:
# --- MAIN DATA GENERATION EXECUTION ---
X_MAX = 1 * 10**7
print(f"Starting data generation up to {X_MAX:,}...")
start_time = time.time()

primes = generate_primes_upto(X_MAX)
pi_metrics = generate_pi_metrics(X_MAX, primes)

end_time = time.time()
print(f"Data generation completed in {end_time - start_time:.2f} seconds.")


Starting data generation up to 10,000,000...
Computing metrics up to N = 10,000,000...
Data generation completed in 0.15 seconds.


In [3]:
def generate_cos_matrix_from_data(fx_values, x_values):
    """
    Generates the harmonic matrix M (cos_matrix) from precomputed vectors.

    This version is significantly more efficient, as it does not compute F(x)
    on the fly.

    Args:
        fx_values (np.ndarray):
            1D array containing the precomputed values of F(x) = |Δπ(x)|.
        x_values (np.ndarray):
            1D array containing the corresponding x values.

    Returns:
        np.ndarray:
            The 2D harmonic matrix M (cos_matrix), ready for visualisation
            and spectral analysis.
    """

    # Ensure input arrays are floating-point for numerical operations
    fx_values = fx_values.astype(np.float64)
    x_values = x_values.astype(np.float64)

    # Safety measure: avoid logarithm of zero or negative values
    x_values[x_values <= 0] = 1e-12  # Replace with a very small positive number

    # Compute the logarithmic scale of x
    logx = np.log(x_values)

    # Core computation using NumPy broadcasting
    # M_ij = cos(F(x_i) * ln(x_j)) + cos(F(x_j) * ln(x_i))
    M = np.cos(np.outer(fx_values, logx)) + np.cos(np.outer(logx, fx_values))

    return M


In [4]:
# (Make sure the functions and the pi_metrics data structure
# have already been executed)

# --- Interactive Plotting Function (Adjusted) ---
def plot_interactive_harmonic_basis(X, N):
    try:
        print(f"Generating harmonic basis for X0={X}, N={N}...")
        
        # Select the window of x-values and corresponding F(x)
        x_vals = pi_metrics['x'][X : X + N]
        fx_vals = pi_metrics['F_x'][X : X + N]
        
        # Build the harmonic matrix M
        M = generate_cos_matrix_from_data(fx_vals, x_vals)
        
        plt.figure(figsize=(8, 8))
        
        # --- MAIN ADJUSTMENT ---
        # 1. Store the object returned by imshow in the variable 'im'
        im = plt.imshow(M, cmap='viridis', origin='lower')
        
        plt.title(f'Harmonic Basis (M) for N={N} at X0={X}')
        plt.xlabel('j')
        plt.ylabel('i')
        
        # 2. Call colorbar ONCE, passing 'im' explicitly
        plt.colorbar(im, fraction=0.046, pad=0.04)
        
        # 3. Show everything at the end
        plt.show()

    except IndexError:
        print(f"Error: X0 out of bounds. The maximum valid value for this N is {len(pi_metrics['x']) - N}.")
    except Exception as e:
        print(f"An error occurred: {e}")

# --- Widget configuration ---
x_slider = widgets.IntSlider(
    value=5_000_000,
    min=1,
    max=len(pi_metrics['x']) - 256,
    step=10_000,
    description='X0:'
)

n_selector = widgets.Dropdown(
    options=[128, 256, 512, 1024],
    value=256,
    description='N:'
)

interactive_plot = widgets.interactive(
    plot_interactive_harmonic_basis,
    X=x_slider,
    N=n_selector
)

print("--- Interactive Harmonic Basis Playground ---")
print("Move the X0 slider and select N to explore different regions of the number line.")
display(interactive_plot)


--- Interactive Harmonic Basis Playground ---
Move the X0 slider and select N to explore different regions of the number line.


interactive(children=(IntSlider(value=5000000, description='X0:', max=9999744, min=1, step=10000), Dropdown(de…

---

> *The human eye is the first detector of symmetry.*  
> *What once appeared as noise begins to reveal form.*

Yet vision, powerful as it is, is only the beginning.  
Visual coherence suggests structure, but it does not measure it.

In the next chapter, we move beyond inspection and apply the formal tools of mathematical physics — **spectral analysis of eigenvalues and eigenvectors** — to  
quantify this emerging harmony.

What will be examined is no longer appearance, but **distribution**. No longer geometry alone, but **statistics of spacings and modes**.

It is there that the hidden signature of the **Gaussian Orthogonal Ensemble (GOE)** emerges — the universal law governing chaotic systems, now arising from a  
**fully deterministic arithmetic construction**.

The spectrometer has shown us *where* to look.  
Spectral analysis will tell us *what* is there.
